In [1]:
#hide
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#hide
%load_ext autoreload
%autoreload 2

%matplotlib inline
%cd /content/gdrive/My Drive/Colab Notebooks

/content/gdrive/My Drive/Colab Notebooks


In [4]:
__all__ =['LabelSmoothingCrossEntropy', 'accuracy','mse','acc_seg','dice_score', 'CrossEntropyFlat', 'DiceLoss','gram_matrix',
          'PerceptualLoss']

In [ ]:
#hide
# !git clone https://github.com/prajwal-suresh13/dl_lib.git

In [5]:
from dl_lib.core.utils import *
from dl_lib.core.callbacks import *

In [ ]:
class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, eta = 0.1, reduction='mean'):
        super().__init__()
        self.eta, self.reduction = eta, reduction

    def forward(self, output, target):
        c = output.size()[-1]
        log_preds = F.log_softmax(output, dim=-1)
        nll = F.nll_loss(log_preds, target, reduction = self.reduction)

        loss = reduce_loss(-log_preds.sum(dim=-1), self.reduction)

        return lin_comb(self.eta, loss/c, nll)

In [ ]:
def mse(output, target): return (output.squeeze(-1) - target).pow(2).mean()

In [ ]:
def accuracy(output, target):
    return (torch.argmax(output, dim=1)==target).float().mean()

In [ ]:
def acc_seg(input, target, ignore_mask=None):
    n=target.shape[0]
    target = target.view(n,-1)
    input = input.argmax(dim=1).view(n,-1)
    if ignore_mask is not None:
        mask = target != ignore_mask
        return (input[mask]==target[mask]).float().mean()
    return (input==target).float().mean()

In [ ]:
def dice_score(pred, target, smooth=1e-6):
    num_of_classes = pred.shape[1]
    actual_classes = [len(target[i].unique()) for i in range(target.shape[0])]
    actual_classes = tensor(actual_classes).cuda()
    # print(num_of_classes, actual_classes)

    targets = _one_hot(target,num_of_classes)
    # print(targets)
    preds = F.softmax(pred,dim=1)
    # print(preds)
    sum_dims = list(range(2,len(pred.shape)))
    # print(sum_dims)
    intersection = torch.sum(preds*targets,dim=sum_dims)
    # print(intersection)
    union = torch.sum(preds+targets, dim=sum_dims)
    dice_score = ((2. * intersection) / (union )).sum(dim=1)
    dice_score/=actual_classes
    return dice_score.mean()

In [ ]:
class CrossEntropyFlat(nn.CrossEntropyLoss):
    def forward(self, input, target):
        n,c,*_ = input.shape
        return super().forward(input.view(n,c,-1), target.view(n,-1))
        #https://discuss.pytorch.org/t/multi-class-semantic-segmentation-using-u-net-error-with-binary-cross-entropy-with-logits/85207

In [ ]:
class DiceLoss(nn.Module):
    def __init__(self, smooth=0.):
        super().__init__()
        self.smooth = smooth

    def forward(self, pred, target):
        dice = dice_score(pred,target, self.smooth)
        return 1-dice

#Perceptual Loss

In [ ]:
#hide
# from torchvision.models import vgg16_bn
# vgg_m = vgg16_bn(True).features.cuda().eval()
# for p in vgg_m.parameters():p.requires_grad_(False)

#hide
# blocks = [i-1 for i,o in enumerate(list(vgg_m.children())) if isinstance(o,nn.MaxPool2d)]
# blocks, [vgg_m[i] for i in blocks]

In [ ]:
def gram_matrix(x):
    n,c,h,w = x.size()
    x = x.view(n, c, -1)
    return (x @ x.transpose(1,2))/(c*h*w)

In [ ]:
class PerceptualLoss(nn.Module):
    def __init__(self, m_feat, layer_ids, layer_wgts, scale=0.01):
        super().__init__()
        self.m_feat = m_feat
        self.loss_features = [self.m_feat[i] for i in layer_ids]
        # self.hooks = Hooks(self.loss_features,hook_outputs)
        self.wgts = layer_wgts
        self.mean = tensor([0.485, 0.456, 0.406]).cuda() 
        self.std = tensor([0.229, 0.224, 0.225]).cuda()
        self.scale=scale
        # self.metric_names = ['pixel',] + [f'feat_{i}' for i in range(len(layer_ids))
        #       ] + [f'gram_{i}' for i in range(len(layer_ids))]

    def make_features(self, x, clone=False):
        with Hooks(self.loss_features, hook_outputs)as hooks:
            self.m_feat(x)
            return [(h.output.clone() if clone else h.output) for h in hooks]
            # return [print(h.output) for h in hooks]
            
    
    def forward(self, input, target):
        input,target= (input/tensor([2.]).cuda())+tensor([0.5]).cuda() ,(target/tensor([2.]).cuda())+tensor([0.5]).cuda()
        input,target = (input-self.mean[...,None,None])/self.std[...,None,None] , (target-self.mean[...,None,None])/self.std[...,None,None]
        out_feat = self.make_features(target, clone=True)
        in_feat = self.make_features(input)
        self.feat_losses = [F.l1_loss(input,target)]
        self.feat_losses += [F.l1_loss(f_in, f_out)*w
                             for f_in, f_out, w in zip(in_feat, out_feat, self.wgts)]
        self.feat_losses += [F.l1_loss(gram_matrix(f_in), gram_matrix(f_out))*w**2 * 5e3
                             for f_in, f_out, w in zip(in_feat, out_feat, self.wgts)]
        # self.metrics = dict(zip(self.metric_names, self.feat_losses))
        self.feat_losses=sum(self.feat_losses) *self.scale
        # print(self.feat_losses)
        return self.feat_losses
    
   

In [6]:
#hide
!pip install fire
!python dl_lib/notebook2script.py notebooks/core/metricsloss.ipynb dl_lib/core

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 87 kB 4.8 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115942 sha256=1d13dd303303539127b6cab91c839e771b19c87764c403616bf20609cc2ca45b
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
Successfully built fire
Converted notebooks/core/metricsloss.ipynb to dl_lib/core/metricsloss.py
